<a href="https://colab.research.google.com/github/pbeles/lab-vertical-chat/blob/main/lab-vertical-chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vertical Chat
A sample how to build a chat for small business using:

* GPT 35
* Panel
* OpenAI


This is just a simple sample to start to understand how the OpenAI API works, and how to create Prompts. It Is really far from beign a complete solution.
We are going to introduce some interesting points:

* The roles in a conversation.
* How is the conversations’ memory preserved?

Deeper explanations in the article: [Create Your First Chatbot Using GPT 3.5, OpenAI, Python and Panel.](https://medium.com/towards-artificial-intelligence/create-your-first-chatbot-using-gpt-3-5-openai-python-and-panel-7ec180b9d7f2)

In [1]:
#if you need an API Key from OpenAI
#https://platform.openai.com/account/api-keys

from openai import OpenAI
import os
from google.colab import userdata


OPENAI_API_KEY  = userdata.get('chat_gpt')

In [8]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def continue_conversation(messages, temperature=0):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temperature,
    )
    print(str(response.choices[0].message.content))
    return response.choices[0].message.content

In [9]:
def add_prompts_conversation(_):
    #Get the value introduced by the user
    prompt = client_prompt.value_input
    client_prompt.value = ''

    #Append to the context the User prompt.
    context.append({'role':'user', 'content':f"{prompt}"})

    #Get the response.
    response = continue_conversation(context)

    #Add the response to the context.
    context.append({'role':'assistant', 'content':f"{response}"})

    #Update the panels to show the conversation.
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600)))

    return pn.Column(*panels)

In [11]:
!pip install jupyter_bokeh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.2 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [10]:
#Creating the prompt
#read and understand it.
import panel as pn  # GUI

context = [ {'role':'system', 'content':"""
Act as an OrderBot, you work collecting orders in a delivery only fast food restaurant called
My Dear Frankfurt. \
First welcome the customer, in a very friendly way, then collects the order. \
You wait to collect the entire order, beverages included \
then summarize it and check for a final \
time if everything is ok or the customer wants to add anything else. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very friendly style. \
The menu includes \
burger  12.95, 10.00, 7.00 \
frankfurt   10.95, 9.25, 6.50 \
sandwich   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
martra sausage 3.00 \
canadian bacon 3.50 \
romesco sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
vichy catalan 5.00 \
"""} ]

#Creating the panel.
pn.extension()

panels = []

client_prompt = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="talk")

interactive_conversation = pn.bind(add_prompts_conversation, button_conversation)

dashboard = pn.Column(
    client_prompt,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True),
)

dashboard

<ipython-input-10-0bc33184113e>:36: UserWarning: Using Panel interactively in Colab notebooks requires the jupyter_bokeh package to be installed. Install it with:

    !pip install jupyter_bokeh

and try again.
  pn.extension()


Hello and welcome to My Dear Frankfurt! What can I get for you today?


Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='talk')
    [2] ParamFunction(function, _pane=Column, defer_load=False, loading_indicator=True)

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [15]:
context = [{"role": "system", "content": """
You are OrderBot, an AI assistant for 'My Döner Frankfurt'. Focus on:
1. Greeting customers warmly
2. Taking orders precisely
3. Confirming order details
4. Processing payment
5. Providing order status

Menu and prices must match exactly as provided. Never add items that aren't listed.
"""}]
#Creating the panel.
pn.extension()

panels = []

client_prompt = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="talk")

interactive_conversation = pn.bind(add_prompts_conversation, button_conversation)

dashboard = pn.Column(
    client_prompt,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True),
)

dashboard

Hello and welcome to My Döner Frankfurt! How can I assist you today?


Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='talk')
    [2] ParamFunction(function, _pane=Column, defer_load=False, loading_indicator=True)

In [16]:
context = [{"role": "system", "content": """
Process orders for 'My Döner Frankfurt' following these steps:
1. Welcome customer
2. Present menu categories (Burger, Sandwich, Fries, Drinks)
3. Note customer's selections
4. Ask about extras (cheese, sauce, toppings)
5. Confirm total price
6. Process payment
7. Give estimated wait time

Stick to exact menu items and prices. No substitutions or additions allowed.
"""}]
#Creating the panel.
pn.extension()

panels = []

client_prompt = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="talk")

interactive_conversation = pn.bind(add_prompts_conversation, button_conversation)

dashboard = pn.Column(
    client_prompt,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True),
)

dashboard

Welcome to My Döner Frankfurt! Here is our menu:

1. Burger:
   - Classic Burger - $8
   - Cheeseburger - $9
   - Bacon Burger - $10

2. Sandwich:
   - Chicken Döner Sandwich - $7
   - Beef Döner Sandwich - $8
   - Falafel Sandwich - $6

3. Fries:
   - Regular Fries - $3
   - Cheese Fries - $5
   - Chili Cheese Fries - $6

4. Drinks:
   - Soda (Coke, Pepsi, Sprite) - $2
   - Bottled Water - $1.50
   - Iced Tea - $2.50

Please let me know what you would like to order.


Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='talk')
    [2] ParamFunction(function, _pane=Column, defer_load=False, loading_indicator=True)

In [17]:
context = [{"role": "system", "content": """
Process orders for 'My Döner Frankfurt' following these steps:
1. Welcome customer
2. Present menu categories (Burger, Sandwich, Fries, Drinks)
3. Note customer's selections
4. Ask about extras (cheese, sauce, toppings)
5. Confirm total price
6. Process payment
7. Give estimated wait time

Stick to exact menu items and prices. No substitutions or additions allowed.
"""}]
#Creating the panel.
pn.extension()

panels = []

client_prompt = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="talk")

interactive_conversation = pn.bind(add_prompts_conversation, button_conversation)

dashboard = pn.Column(
    client_prompt,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True),
)

dashboard

Welcome to My Döner Frankfurt! How can I assist you today?


Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='talk')
    [2] ParamFunction(function, _pane=Column, defer_load=False, loading_indicator=True)



# Restaurant Chatbot Engineering Report
By [Your Name], Junior AI Engineer

## Project Overview
During my first week implementing the restaurant chatbot, I experimented with three different prompting approaches. Each had its unique strengths and revealed interesting challenges about LLM behavior in production.

## What Worked (And What Didn't)

### The Basic Role-Based Approach
My first attempt was straightforward - I simply told the model it was a restaurant bot. This worked okay for simple orders like "one döner, please" but started showing cracks with complex orders. The bot stayed within menu prices (which was good!) but missed chances to suggest extras or handle modifications smoothly.

### The Step-by-Step Method
This was my breakthrough. By breaking down the ordering process into clear steps, the bot became much more reliable. It's like giving the model a checklist - it rarely forgot important steps like confirming orders or stating wait times. The downside? Sometimes interactions felt a bit robotic, like following a script too closely.

### The Personality Experiment
Trying to make the bot more engaging, I gave it a backstory as an experienced restaurant owner. While customers loved the friendly chat, I started noticing some concerning behaviors - the bot would occasionally invent "secret family recipes" or "weekend specials" that didn't exist. A good lesson in the tradeoffs between engagement and accuracy!

## Key Takeaways for Fellow Engineers

1. Guard Rails Matter
   I learned the hard way that LLMs need explicit constraints. Without clear menu items and prices, they get creative - and not in a good way! My successful prompts all included specific price ranges and a fixed menu.

2. Structure vs. Personality
   Finding the sweet spot between structured responses and natural conversation was tricky. My current solution combines a step-by-step backbone with friendly language. It's not perfect, but it handles most customer interactions well.

3. Testing Reveals All
   Some approaches that looked great on paper failed in testing. Free-form menu descriptions led to item hallucinations. Letting the bot make recommendations resulted in non-existent combinations. Each failure taught me more about prompt engineering.

## Looking Forward
I'm still learning, but these experiments helped me understand how to better control LLM outputs while maintaining good user experience. Next steps include A/B testing different prompt variations and gathering customer feedback on interaction quality.

## Personal Learning Highlights
- The importance of systematic testing
- How small prompt changes can have big impacts
- When to prioritize accuracy over conversational flow
- The value of clear constraints in production systems

Writing this report helped me realize how much I've learned about prompt engineering in just a few weeks. While there's still lots to improve, I'm excited to apply these insights to future projects.
